# 3.2 Screenshot for 3x2 matrix

## Policy Calculation

### Derivation of Probabilities

In [18]:
import copy

with open ("SAKI Exercise 3 warehousetraining2x2.txt","r") as inputpar:
    trainingSet = inputpar.readlines()
    
cleaned_training = []
for i, x in enumerate(trainingSet):
    y = x.replace("\t", "")
    y = y.replace("\n", "")
    cleaned_training.append(y)

# Probabilities for each subsequent state
listStoreWhite =[]
listStoreRed = []
listStoreBlue = []
listRestoreWhite = []
listRestoreRed = []
listRestoreBlue = []
together = [listStoreRed,  listStoreBlue, listStoreWhite, listRestoreRed, listRestoreBlue, listRestoreWhite,]
for i, entry in enumerate(cleaned_training):
    if i < len(cleaned_training)-1:
        if entry == "storered":
                listStoreRed.append(cleaned_training[i+1])
        elif entry == "storeblue":
                listStoreBlue.append(cleaned_training[i+1])
        elif entry == "storewhite":
                listStoreWhite.append(cleaned_training[i+1])
        elif entry == "restorered":
                listRestoreRed.append(cleaned_training[i+1])
        elif entry == "restoreblue":
                listRestoreBlue.append(cleaned_training[i+1])
        elif entry == "restorewhite":
                listRestoreWhite.append(cleaned_training[i+1])

action_col_combination = []
colour = ["red", "blue", "white"]
store_restore = ["store", "restore"]
for act in store_restore:
        for col in colour:
            action_col_combination.append(act+col)

anzahlDict = {}
trainingsDict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
sums = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

for i, x in enumerate(together):
        for ind in action_col_combination:
            anzahlDict[ind] = x.count(ind)
        for summe in anzahlDict:
            sums[i] += anzahlDict[summe]
        trainingsDict[i] = copy.deepcopy(anzahlDict)

percentageDict = copy.deepcopy(trainingsDict)

# Derivation of probabilites
for i, o in enumerate(trainingsDict):
    for x in action_col_combination:
        percentageDict[i][x] = trainingsDict[i][x]/sums[i]
for i in range(0, 6):
    percentageDict[action_col_combination[i]] = percentageDict[i]
    del percentageDict[i]

## Setting up State-matrix

In [19]:
import itertools

fields = [(1, 1), (1, 2), (2, 1), (2, 2), (3,1), (3,2)]

field_combination = []
for i in range(0, len(fields)+1):
    field_combination.append(list(itertools.combinations(fields, i)))
    

def combinations_with_replacement(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    for indices in itertools.product(range(n), repeat=r):
        yield tuple(pool[i] for i in indices)

sr_space = []
for act in store_restore:
    for col in colour:
        sr_space.append([act, col])

color_combinations_state = []
for i in range(0, len(fields)+1):
        color_combinations_state.append(list(combinations_with_replacement(colour, i)))

final_states = []
for sr in sr_space:
        for i, field in enumerate(field_combination):
                for entry in field:
                        for col in color_combinations_state[i]:
                                final_states.append((entry, col, sr))                                
print(len(final_states))

24576


## Calculating Transition Probability Matrix and Reward Matrix

In [21]:
import numpy as np
import pandas as pd
import csv

lookup_mtrx = {(1, 1): 8, (1, 2): 6, (2, 1): 6, (2, 2): 4, (3, 1): 4, (3, 2): 2}

trans_prob_matrix = []
reward_matrix = []
for i in range(len(fields)):
        trans_prob = np.zeros((len(final_states), len(final_states)), dtype=np.float16)
        reward = np.full((len(final_states), len(final_states)), -6, dtype=np.int8)
        trans_prob_matrix.append(trans_prob)
        reward_matrix.append(reward)
print("matrix intialized")
print(len(trans_prob_matrix[0]))


def FindNewCoordinate(stateA, stateB):
        a = 0

        stateA = list(map(list, stateA))
        stateB = list(map(list, stateB))

        while a != len(stateA[0]):
                coordA = stateA[0][a]
                if coordA in stateB[0]:
                        indexB = stateB[0].index(coordA)
                        if stateA[1][a] == stateB[1][indexB]:
                                del stateA[0][a]
                                del stateB[0][indexB]
                                del stateA[1][a]
                                del stateB[1][indexB]
                                a -= 1
                a += 1

        if len(stateA[0]) == 0:
                stateB[0] = tuple(stateB[0])
                stateB[1] = tuple(stateB[1])
                return stateB, len(stateB[0])
        elif len(stateB[0]) == 0:
                stateA[0] = tuple(stateA[0])
                stateA[1] = tuple(stateA[1])
                return stateA, len(stateA[0])
        return False


# set up transition probability and reward matrix
for a, action in enumerate(fields):
        print(action)
        for y, row in enumerate(final_states):
                potential_illegal_states = []
                for x, col in enumerate(final_states):
                        if row[2][0] == "store":
                                if len(row[0])+1 == len(col[0]):
                                        helfer = FindNewCoordinate(row, col)
                                        if helfer is not False:
                                                if helfer[0][0][0] == action and helfer[0][1][0] == row[2][1]:
                                                        trans_prob_matrix[a][y][x] = percentageDict[row[2][0]+row[2][1]][col[2][0]+col[2][1]]
                                                        reward_matrix[a][y][x] = lookup_mtrx[action]
                                elif len(row[0]) == len(col[0]):
                                        helfer = FindNewCoordinate(row, col)
                                        if helfer is not False and helfer[1] == 0:
                                                potential_illegal_states.append(x)

                        if row[2][0] == "restore":
                                if len(row[0])-1 == len(col[0]):
                                        helfer = FindNewCoordinate(row, col)
                                        if helfer is not False:
                                                if helfer[0][0][0] == action and helfer[0][1][0] == row[2][1]:
                                                        trans_prob_matrix[a][y][x] = percentageDict[row[2][0]+row[2][1]][col[2][0]+col[2][1]]
                                                        reward_matrix[a][y][x] = lookup_mtrx[action]
                                elif len(row[0]) == len(col[0]):
                                        helfer = FindNewCoordinate(row, col)
                                        if helfer is not False and helfer[1] == 0:
                                                potential_illegal_states.append(x)

                if np.sum(trans_prob_matrix[a][y], dtype=np.float16) == 0:
                        assert len(potential_illegal_states) == 6
                        for candidates in potential_illegal_states:
                                 trans_prob_matrix[a][y][candidates] = percentageDict[row[2][0]+row[2][1]][final_states[candidates][2][0]+final_states[candidates][2][1]]

                if np.sum(trans_prob_matrix[a][y], dtype=np.float16) != 1:
                        tmp = np.float16(1.0)-np.sum(trans_prob_matrix[a][y], dtype=np.float16)
                        can = np.argwhere(x > 0)
                        trans_prob_matrix[a][y][can[-1]] += tmp
                assert np.sum(trans_prob_matrix[a][y], dtype=np.float16) == 1.0

        #pd.DataFrame(trans_prob_matrix[a]).to_csv(str(action)+"transProbMatrix_adap_2x2.csv", sep=";", header=final_states, decimal=".")
        #pd.DataFrame(reward_matrix[a]).to_csv(str(action)+"rewardMatrix_adap_2x2.csv", sep=";", header=final_states, decimal=".")
        print("end of action")
print("end of transprob and reward matrix")


matrix intialized
24576
(1, 1)
end of action
(1, 2)
end of action
(2, 1)
end of action
(2, 2)
end of action
(3, 1)
end of action
(3, 2)
end of action
end of transprob and reward matrix


## Policy Derivation using MDP Toolbox

In [5]:
!pip install pymdptoolbox

smart-open 1.6.0 requires bz2file, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [14]:
import pandas as pd
import time
ts_beginning = time.time()
pi_vigs = mdptoolbox.mdp.RelativeValueIteration(trans_prob_matrix, reward_matrix)
pi_vigs.run()
x = list(pi_vigs.policy)
pd.DataFrame(x).to_csv("RelativeValueIteration3x2.csv", sep=";")
ts_end = time.time()
time_difference = ts_end-ts_beginning
print(time_difference)
pi_vigs.iter

3521.594519853592


90

In [15]:
import mdptoolbox
ts_beginning = time.time()
pi_vigs = mdptoolbox.mdp.ValueIteration(trans_prob_matrix, reward_matrix, 0.99)
pi_vigs.run()
pd.DataFrame(list(pi_vigs.policy)).to_csv("policyValueIteration0.993x2.csv", sep=";")
ts_end = time.time()
time_difference = ts_beginning - ts_end
print(time_difference)
pi_vigs.iter

-8178.164109706879


209

In [16]:
pi_vigs = mdptoolbox.mdp.ValueIterationGS(trans_prob_matrix, reward_matrix, 0.999)
pi_vigs.run()
pd.DataFrame(pi_vigs.policy).to_csv("policyValueIterationGS3x2.csv", sep=";")
pi_vigs.iter

KeyboardInterrupt: 

# Policy Evaluation

## Import Warehouse Order

In [ ]:
with open ("SAKI Exercise 3 warehouseordernew.txt","r") as inputpar:
    trainingSet = inputpar.readlines()

iter_trainingsset= []
for i, x in enumerate(trainingSet):
        y = x.replace("\t", " ")
        y = y.replace("\n", "")
        y = y.split()
        iter_trainingsset.append(y)

## Greedy Algorithm

In [ ]:
distance_greedy = {(1, 1): 1, (1, 2): 2, (2, 1): 2, (2, 2): 3, (3, 1): 3, (3, 2): 4}
greedy_matrix = [[0, "empty"], [0, "empty"], [0, "empty"], [0, "empty"], [0, "empty"], [0, "empty"]]
eval_greedy_algorithm = []
for row in iter_trainingsset:
    if row[0] == "store":
        for i, action in enumerate(fields):
            if greedy_matrix[i][0] == 0:
                greedy_matrix[i][0] = 1
                greedy_matrix[i][1] = row[1]
                eval_greedy_algorithm.append([row, action, distance_greedy[action]])
                break
    elif row[0] == "restore":        
        for i, action in enumerate(fields):
            if greedy_matrix[i][0] == 1 and row[1] == greedy_matrix[i][1]:
                greedy_matrix[i][0] = 0
                greedy_matrix[i][1] = "empty"
                eval_greedy_algorithm.append([row, action, distance_greedy[action]])
                break
pd.DataFrame(eval_greedy_algorithm).to_csv("eval_greedy_algorithm.csv", sep=";")

print(eval_greedy_algorithm)


## Evaluation of MDP policies

In [ ]:
import re

with open("ModifiedPolicyIteration.csv", "r") as inputpar:
    policy = inputpar.readlines()

cleaned_policy = []
for i, x in enumerate(policy):
    if i > 0:
        if len(cleaned_policy) == 9 or len(cleaned_policy) == 99 or len(cleaned_policy) == 999 or len(cleaned_policy) == 9999:
            y = x[len(str(i)):]
            y = y.replace("\n", "")
            cleaned_policy.append(y)
        else:
            y = x[len(str(i))+1:]
            y = y.replace("\n", "")
            cleaned_policy.append(y)


def lookup_candidate(matrix, stateB):
    list_coord = []
    list_colour = []
    for i in matrix:
        if i[1] != "empty":
            list_coord.append(i[2])
            list_colour.append(i[1])
    if len(list_coord) != len(stateB[0]):
        return False
    for i in range(0, len(list_coord)):
        if list_coord[i] != stateB[0][i] or list_colour[i] != stateB[1][i]:
            return False
    return True


rl_matrix = [[0, "empty", (1, 1)], [0, "empty", (1, 2)], [0, "empty", (2, 1)], [0, "empty", (2, 2)], [0, "empty", (3, 1)], [0, "empty", (3, 2)]]
distance_rl = {"0": 1, "1": 2, "2": 3, "3": 2, "4": 3, "5": 4}
rl_algorithm = []


def empty_counter(matrix):
    counter = 0
    for entry in rl_matrix:
        if entry[1] != "empty":
            counter += 1
    return counter


def inner_func():
    for state_index, row_states in enumerate(final_states):
            if row_states[2][0] == row[0] and row_states[2][1] == row[1]:
                            if not(rl_matrix[0][1] == rl_matrix[1][1] == rl_matrix[2][1] == rl_matrix[3][1] == "empty"):
                                    count = 0
                                    for it in range(0, len(row_states[0])):
                                            if rl_matrix[it][1] == "empty" and row_states[1][it] != "":
                                                    if lookup_candidate(rl_matrix, row_states) is True and row[0] == "store":
                                                                    step_counter = str(cleaned_policy[state_index])
                                                                    rl_matrix[int(cleaned_policy[state_index])][0] = 1
                                                                    rl_matrix[int(cleaned_policy[state_index])][1] = row[1]
                                                                    rl_algorithm.append([row, rl_matrix[int(cleaned_policy[state_index])][2], distance_rl[step_counter]])
                                                                    return
                                                    if lookup_candidate(rl_matrix, row_states) is True and row[0] == "restore":
                                                                    step_counter = str(cleaned_policy[state_index])
                                                                    rl_matrix[int(cleaned_policy[state_index])][0]= 0
                                                                    rl_matrix[int(cleaned_policy[state_index])][1]="empty"
                                                                    rl_algorithm.append([row,rl_matrix[int(cleaned_policy[state_index])][2],distance_rl[step_counter]])
                                                                    return
                                                    else:
                                                            continue                                                                                                                      
                                            if rl_matrix[it][2] == row_states[0][it] and rl_matrix[it][1] == row_states[1][it]:
                                                    count += 1
                                                    if count == len(row_states[0]) == empty_counter(rl_matrix):
                                                            if row[0] == "store":
                                                                    step_counter = str(cleaned_policy[state_index])
                                                                    rl_matrix[int(cleaned_policy[state_index])][0] = 1
                                                                    rl_matrix[int(cleaned_policy[state_index])][1] = row[1]
                                                                    rl_algorithm.append([row, rl_matrix[int(cleaned_policy[state_index])][2], distance_rl[step_counter]])
                                                                    return
                                                            elif row[0] == "restore":
                                                                    step_counter = str(cleaned_policy[state_index])
                                                                    rl_matrix[int(cleaned_policy[state_index])][0] = 0
                                                                    rl_matrix[int(cleaned_policy[state_index])][1] ="empty"
                                                                    rl_algorithm.append([row, rl_matrix[int(cleaned_policy[state_index])][2], distance_rl[step_counter]])
                                                                    return
                            # case empty matrix
                            else:
                                    if row[0] == row_states[2][0] and row[1] == row_states[2][1]:
                                            if row[0] == "store":
                                                    step_counter = str(cleaned_policy[state_index])
                                                    rl_matrix[int(cleaned_policy[state_index])][0] = 1
                                                    rl_matrix[int(cleaned_policy[state_index])][1] = row[1]
                                                    rl_algorithm.append([row, rl_matrix[int(cleaned_policy[state_index])][2], distance_rl[step_counter]])
                                                    return
                                            elif row[0] == "restore":
                                                    step_counter = str(cleaned_policy[state_index])
                                                    rl_matrix[int(cleaned_policy[state_index])][0] = 0
                                                    rl_matrix[int(cleaned_policy[state_index])][1] = "empty"
                                                    rl_algorithm.append([row, rl_matrix[int(cleaned_policy[state_index])][2], distance_rl[step_counter]])
                                                    return

for j, row in enumerate(iter_trainingsset):
    inner_func()

pd.DataFrame(rl_algorithm).to_csv("rl_algorithm.csv", sep=";")
print(rl_algorithm)